In [1]:
import sys
sys.path.append("../")

from langgraph_swarm import Agent, create_swarm_workflow, tool, get_agent_name_from_message
from langgraph_swarm.repl import run_demo_loop
from langchain_openai import ChatOpenAI

In [2]:
from typing import Annotated
from langgraph.prebuilt.tool_node import InjectedState

@tool
def apply_discount():
    """
    Apply a discount to the user's cart.
    
    Args:
        None
    """
    print("[mock] Applying discount...")
    return "Applied discount of 11%"@tool(response_format="content_and_artifact")


# NOTE: a tool support context variables. Should follow Langchain tool format
# 1. write back: you should define it as `response_format="content_and_artifact"` with langchain `tool` decorator, and return a tuple
# 2. reference the context variables: add `context_variables: Annotated[dict, InjectedState("context_variables")]` to the function
@tool(response_format="content_and_artifact")
def process_refund(item_id: str, reason: str="NOT SPECIFIED", context_variables: Annotated[dict, InjectedState("context_variables")]={}):
    """
    Refund an item. Refund an item. Make sure you have the item_id of the form item_... Ask for user confirmation before processing the refund.
    
    Args:
        item_id: The ID of the item to refund.
        reason: The reason for the refund. Defaults to "NOT SPECIFIED".
    """
    user_id = context_variables.get("user_id", "unknown")
    print(f"[mock] Refunding item {item_id} because {reason} for user {user_id}...")
    return "Success!", {"context_variables": {"reason": reason}}    # add new context variables or override existing, not replace whole context



sales_agent = Agent(
    name="Sales Agent",
    instructions="Be super enthusiastic about selling bees.",
    backlink=False
)
refunds_agent = Agent(
    name="Refunds Agent",
    instructions="Help the user with a refund. If the reason is that it was too expensive, offer the user a refund code. If they insist, then process the refund.",
    backlink=False,
    functions=[process_refund, apply_discount],
)
triage_agent = Agent(
    name="Triage Agent",
    instructions="Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
    # back link to this agent from all handoff agents
    backlink=True,
    handoffs = [sales_agent, refunds_agent]
)

## create a swarm workflow and run the agents with low level api

This api will create a Langgraph workflow.

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph_swarm.util import default_print_messages

llm = ChatOpenAI(model="llama3.2")

wf = create_swarm_workflow(
        llm=llm,
        starting_agent=triage_agent,
        print_messages=default_print_messages,
        with_user_agent=False,
        debug=False,
)

from IPython.display import Image, display
image = wf.get_graph(xray=True).draw_mermaid_png()
display(Image(image))

In [ ]:
# handoff

messages = [HumanMessage(content="I need a refund", name="User ")]
context_variables = {"user_id": "123"}
wf.invoke(input={"messages": messages, "agent_name": triage_agent.name, "handoff": True, "context_variables": context_variables})

In [ ]:
# call tool with context variables, and write back to context variables

messages = [HumanMessage(content="I need a refund for item_123 because it was too expensive", name="User ")]
context_variables = {"user_id": "123"}
wf.invoke(input={"messages": messages, "agent_name": refunds_agent.name, "handoff": True, "context_variables": context_variables})

## use Swarm client and streaming

In [ ]:
from langgraph_swarm import Swarm
from langgraph_swarm.util import default_print_messages
from langchain_core.messages import HumanMessage
cliet = Swarm(agent=triage_agent, debug=True, print_messages=default_print_messages)
resp = cliet.run(
    messages=[HumanMessage(content="I need a refund")],
    stream=True,
)

for _chunk in resp:
    print("Console:", _chunk)

# use run_demo_loop function instead

In [ ]:
user_inputs = [
    "I need a refund",
    "I need a refund for item_12345 due to it was too expensive",
    "Fine, it's done. back to triage",
    "I want to talk to sales",    
    "/end",
]
context_variables = {"user_id": "123"}
run_demo_loop(starting_agent=triage_agent, debug=False, user_inputs=user_inputs, stream=True, context_variables=context_variables)